# Azure AI Search RAG Challenge

Scenario: Chat with your product manuals. Product manuals are stored in PDF.

Steps in this challenge:
0. Setup Azure AI Search, Azure OpenAI, and Azure Storage
1. Create an index named product-index. 
  - Fields are title, content, and contentVector.
  - Vector size is 3072 using text embedding 3 large
2. Create a data source named product-ds. 
  - Upload product files into a blob storage
3. Create a skillset named product-ss
  - Split large documents
  - Compute embedding
4. Create an indexer named product-indxr
  - No schedule needed.
5. Perform a semantic hybrid search to test
  - Ask questions from the product files
6. Send query results to a language model to generate response
  - gpt-4o model
  - Send top 5 documents retrieved to language model

## Load Azure Configurations

In [10]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-10-01-preview"
azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_search_service_index_name = "product-index"
azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
azure_ai_services_key = os.getenv("AZURE_AI_MULTISERVICE_KEY")

## Create index

In [11]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    ScoringProfile,
    TagScoringFunction,
    TagScoringParameters
)

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)

# Search index name  
index_name = azure_search_service_index_name

# Create a Search Index Client
index_client = SearchIndexClient(endpoint=azure_search_service_endpoint, credential=credential)

# Define the fields collection
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=3072, vector_search_profile_name="myHnswProfile")
    ]  
  
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[   # a vectorizer is software that performs vectorization
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=azure_openai_endpoint,  
                deployment_name=azure_openai_embeddings_deployment,
                model_name=azure_openai_embeddings_deployment
            ),
        ),  
    ], 
)  

# New semantic configuration
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="chunk")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index
index = SearchIndex(name=index_name, 
                    fields=fields, 
                    vector_search=vector_search,
                    semantic_search=semantic_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")

product-index created


## Create data source

In [12]:
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

# Create a data source 
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)
container = SearchIndexerDataContainer(name="products")
data_source_connection = SearchIndexerDataSourceConnection(
    name="products-ds",
    type="azureblob",
    connection_string=azure_storage_connection_string,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'products-ds' created or updated


## Create a skillset

In [13]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset,
    CognitiveServicesAccountKey
)

# Create a skillset  
skillset_name = "product-ss"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=2000,  
    page_overlap_length=500,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/content"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_url=azure_openai_endpoint,  
    deployment_name=azure_openai_embeddings_deployment,  
    model_name=azure_openai_embeddings_deployment,
    dimensions=3072,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)
  
index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=azure_search_service_index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),  
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

cognitive_services_account = CognitiveServicesAccountKey(key=azure_ai_services_key)

skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents, generate embeddings",  
    skills=skills,  
    index_projection=index_projections,
    cognitive_services_account=cognitive_services_account
)
  
client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")  

product-ss created


## Create indexer

In [14]:
from azure.search.documents.indexes.models import (
    SearchIndexer,
    IndexingSchedule
)

# Create an indexer  
indexer_name = "product-idxr" 

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents, generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,
    parameters=indexer_parameters
)  

# Create and run the indexer  
indexer_client = SearchIndexerClient(endpoint=azure_search_service_endpoint, credential=credential)  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')  

 product-idxr is created and running. Give the indexer a few minutes before running a query.


## Perform a Semantic Hybrid Search

In [16]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# User Query
query = "What is the price of the Trailmaster X4 Tent?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk"],
    top=5,
)

# Sort the results by score in descending order
sorted_results = sorted(results, key=lambda x: x['@search.score'], reverse=True)

for result in sorted_results:  
    print(f"Score: {result['@search.score']} \n")
    print(f"Title: {result['title']} \n")
    print(f"Chunk: {result['chunk']} \n")


Score: 0.03306011110544205 

Title: product_info_1.pdf 

Chunk: # Information about product item_number: 1
TrailMaster X4 Tent, price $250,

## Brand
OutdoorLiving

## Category
Tents

## Features
- Polyester material for durability
- Spacious interior to accommodate multiple people
- Easy setup with included instructions
- Water-resistant construction to withstand light rain
- Mesh panels for ventilation and insect protection
- Rainfly included for added weather protection
- Multiple doors for convenient entry and exit
- Interior pockets for organizing small items
- Reflective guy lines for improved visibility at night
- Freestanding design for easy setup and relocation
- Carry bag included for convenient storage and transportation

## Technical Specs
**Best Use**: Camping  
**Capacity**: 4-person  
**Season Rating**: 3-season  
**Setup**: Freestanding  
**Material**: Polyester  
**Waterproof**: Yes  
**Floor Area**: 80 square feet  
**Peak Height**: 6 feet  
**Number of Doors**: 2  
*

## Send query results to a language model to generate response

In [17]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# Azure OpenAI client
openai_client = AzureOpenAI(
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key)

# Provide instructions to the model
SYSTEM_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# User Query
query = "What is the warranty of the Trailmaster X4 tent?"  

# Convert query into vector form
vector_query = VectorizableTextQuery(text=query, 
                                     k_nearest_neighbors=50, 
                                     fields="text_vector",
                                     weight=1)

results = search_client.search(
    query_type="semantic", 
    semantic_configuration_name='my-semantic-config',
    search_text=query,
    vector_queries= [vector_query],
    select=["title","chunk"],
    top=5,
)

# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=azure_openai_deployment
)

print(response.choices[0].message.content)


The TrailMaster X4 Tent comes with a 2-year limited warranty from the date of purchase. This warranty covers manufacturing defects in materials and workmanship, including seam or fabric tears under normal use, issues with tent poles, zippers, buckles, and problems with the rainfly or other included accessories that affect the performance of the tent. Damage caused by misuse, abuse, improper care, or extreme weather conditions is not covered. (Source: product_info_1.pdf)
